In [ ]:
import requests
import pandas as pd
import numpy as np
import geocoder
import re
import os
import time


from tqdm import tnrange, tqdm_notebook, tqdm_pandas
from tqdm import tqdm
tqdm.pandas(desc="p")

import pickle

In [ ]:
# this complains it's deprecated, but actually is the only thing that's worked and not outputted tons of lines for df.progress_apply 
# I think? I think this was the command that caused it, I deleted it because of the warning initially and didn't realize it hard worked
# todo add version numbers to python packages in dockerifle
tqdm_pandas(tqdm())

In [ ]:
providers = ['google', 'bing', 'mapbox']
providers[0]

In [ ]:
provider_index = 0
# not used currently
def get_from_provider(x):
    current = providers[provider_index]
    if current is 'google':
        return geocoder.google(x, key=os.environ['GOOGLE_API_KEY'])
    if current is 'bing':
        return geocoder.bing(x, key=os.environ['BING_API_KEY'])
    if current is 'mapbox':
        return geocoder.mapbox(x, key=os.environ['MAPBOX_ACCESS_TOKEN'])    
        

In [ ]:
# cache place queries  latlng 
# don't execute this unless you want to reset the cache, or the first time XXX
geo_cache = {"cache_hit_count":0, "cache_miss_count":0}

In [506]:

# take a string, get a latlng, with this simple cache
def get_lat_lng(x):
    debug = False
    # debug = True
    if debug: print('\n'+x)
    # do we have it and is it not from an over limit sitch
    if x in geo_cache and geo_cache[x] != ['', ''] :
        geo_cache["cache_hit_count"]+=1
        res = geo_cache[x]
        return res
    g = geocoder.google(x) # g= get_from_provider(x) 
    geo_cache["cache_miss_count"]+=1    
    if g.latlng:
        geo_cache[x] = g.latlng
        res = g.latlng
    else:
        if g.json['status'] == "ZERO_RESULTS": # if g.json['status_code'] is 200:
            res = [pd.np.nan, pd.np.nan]
            # cache this too!
            geo_cache[x] = res
        else:
            # 'OVER_QUERY_LIMIT'
            print('Problem!!!')
            print(g.json)
            if g.json['status'] == OVER_QUERY_LIMIT:
                raise Exception('Over the query limit')
            res = ['', '']
            # let's stick with google for now, otherwise do
            # provider_index+=1
    if debug: print(res)
    # this is silly but...
    time.sleep(0.15)
    return res

In [277]:
# woof non-google ones are pretty far  off?
# print(get_lat_lng('Brownstone Bagel, Brooklyn, NY'))
# provider_index+=1
# geo_cache = {}
# print(get_lat_lng('Brownstone Bagel, Brooklyn, NY'))
# provider_index+=1
# geo_cache = {}
# print(get_lat_lng('Brownstone Bagel, Brooklyn, NY'))
# get_lat_lng('Brownstone Bagel, Brooklyn, NY')

get_lat_lng('Brooklyn, NY')

[40.6781784, -73.9441579]

In [507]:
# uses above, broadening search if it can't find a business or address
def get_loc_for_row(row, key): 
    debug = False
    place = row[key]
    # place = row.employer
    # try it with no processing first
    if debug: print('Checking\n'+place)
    loc = get_lat_lng(place)
    if loc[0] is not pd.np.NaN:
        return loc
    # then try it dropping a possible business name, or whatever is before the first comma
    if place.find(',') is not -1:
        no_b_name = ','.join(place.split(',')[1:]).strip()
        if debug: print(no_b_name)
        loc = get_lat_lng(no_b_name)
        if loc[0] is not pd.np.NaN:
            return loc        
    m = re.search("\d", place)
    # todo, could do same as above, with another drop. TODO make sure this is all working first XXX
    # now drop everything before the first digit and try that
    if m:
        after_digit = place[m.start():]
        if debug: print(after_digit)
        loc = get_lat_lng(after_digit)
        if loc[0] is not pd.np.NaN:
            return loc  
    # this should be NaN, NaN if we failed to find something, from the first one at least
    return loc 

In [464]:
# works
# get_loc_for_row({'k':'Terry"s Construction, Blackwell, OK 74631'}, 'k')

Checking
Terry"s Construction, Blackwell, OK 74631
Blackwell, OK 74631


[36.8044767, -97.2828207]

### OSHA Data Part 1: XML of recent fatalities and catastrophe
The recent stuff is in xml, the older stuff is csv, don't worry, they don't have the same fields either

In [278]:
# https://stackoverflow.com/a/10077069/83859
from xml.etree import ElementTree
from collections import defaultdict

def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k:v[0] if len(v) == 1 else v for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v) for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
              d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [279]:
response = requests.get('https://www.osha.gov/dep/fatcat/fatcats.xml')
# print(response)

tree = ElementTree.fromstring(response.content)

In [280]:
pd.set_option('max_colwidth', 800)


In [281]:


# this has 2016 and 2017, in xml
osha_current = pd.DataFrame.from_records( etree_to_dict(tree)['events']['incident'] )

In [ ]:
# osha_current.drop(['fatcat', 'inspection', 'victim'], axis=1, inplace=True)

In [282]:
osha_current[-4:]

,date,description,employer,fatcat,inspection,victim
569,2016-06-27,Worker died from hantavirus after being exposed to mouse droppings while cleaning cabins.,"Marzano Mountain Retreat, Torreon, NM 87061",Fatality,1154167,Shawnee Romero
570,2016-05-24,Worker struck and killed by falling tree.,"Raymond Trent dba Raymond Trent, Morganfield, KY 42437",Fatality,1186003.015,James Finwick
571,None,Worker died after being struck by forklift.,"Genuine Parts Company, 30-03 Review Ave LONG ISLAND CITY NY 11101",Fatality,1209887,None
572,None,Worker fatally crushed between tow truck and another vehicle.,Berkebile Auto Service 10 D Street JOHNSTOWN PA 15906,Fatality,1197414,James Harr


In [ ]:
    # this is the one from the ny map, needs to be broader now
    # just try it straight?
    # actually XXX for deathmap, should use the [1:] method!
#     return get_lat_lng(row.employer.split(',')[-1].strip())
#     "MHP Real Estate Services, 180 Maiden Lane NEW YO".split(',')[-1].strip()
# osha_current[lat]

# get_osha_current_loc("Genuine Parts Company 30-03 Review Ave LONG ISLAND CITY NY 11101")
# tqdm_notebook().pandas()
osha_current_loc = osha_current.progress_apply (lambda row: get_loc_for_row(row, 'employer'),axis=1)
#.apply(pd.Series)

In [294]:
osha_current_loc = osha_current_loc.apply(pd.Series) 

In [ ]:
# p bar working example blah
# for i in tnrange(10, desc='1st loop'):
#     for j in tqdm_notebook(range(100), desc='2nd loop'):
#         time.sleep(0.01)

In [ ]:
# tqdm_notebook().pandas(*args, **kwargs)
# https://stackoverflow.com/questions/40476680/how-to-use-tqdm-with-pandas-in-a-jupyter-notebook

In [295]:
# osha_current_loc[osha_current_loc.duplicated(keep=False)]
osha_current['lat'] = osha_current_loc[0]
osha_current['lng'] = osha_current_loc[1]
osha_current[:1]

,date,description,employer,fatcat,inspection,victim,lat,lng
0,2017-04-28,A worker died after being caught between two vehicles.,"PROWLER RECOVERY, INC., 8100 W. Broward Blvd. PLANTATION FL 33324",Fatality,1229035.015,Francisco Canjura,26.122348,-80.258951


In [296]:
osha_current.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573 entries, 0 to 572
Data columns (total 8 columns):
date           571 non-null object
description    573 non-null object
employer       573 non-null object
fatcat         573 non-null object
inspection     551 non-null object
victim         537 non-null object
lat            561 non-null float64
lng            561 non-null float64
dtypes: float64(2), object(6)
memory usage: 35.9+ KB


In [ ]:
# there's really no reason to use a date-index, and we'll lose some "fy 06" stuff from the archives
# osha_current['date']= pd.to_datetime(osha_current['date'])
# osha_current = osha_current.set_index((osha_current.select_dtypes(include=[np.datetime64]).columns).tolist())
# osha_current.sort_index(inplace=True)
osha_current[:3]
# ok next combine that with the other osha stuff

In [297]:
# osha_current[-3:]
osha_current.to_csv('osha_current.csv')
# todo, standardize before dropping stuff

In [ ]:
osha_current.info()

### OSHA Part 2, Archive CSVs 2015 and Before

In [ ]:
osha_archive_urls = [
"https://www.osha.gov/dep/fatcat/FatalitiesFY09.csv",
"https://www.osha.gov/dep/fatcat/FatalitiesFY10.csv",
"https://www.osha.gov/dep/fatcat/FatalitiesFY11.csv",
"https://www.osha.gov/dep/fatcat/fy13_federal-state_summaries.csv",
"https://www.osha.gov/dep/fatcat/fy14_federal-state_summaries.csv",
"https://www.osha.gov/dep/fatcat/fy15_federal-state_summaries.csv"]

# "https://www.osha.gov/dep/fatcat/FatalitiesFY12.csv",

# TODO START HERE AGAIN, fix them on the way in, not later

osha_archive = pd.DataFrame()
for url in tqdm(osha_archive_urls):
    df = pd.DataFrame.from_csv(url)
    print('\nurl',url)
    print(df.info())
    osha_archive = pd.concat([osha_archive, df])
    

In [474]:
osha_archive.dropna(axis=1, how='all', inplace=True)

In [486]:
# do 2012 by itself because it's broken a bit
twelve = pd.DataFrame.from_csv("https://www.osha.gov/dep/fatcat/FatalitiesFY12.csv")
twelve.columns = ['Summary Report Date', 'Date of Incident', 'Company', 'Preliminary Description of Incident']
# twelve["Unnamed: 4"].name = "Preliminary Description of Incident"
# twelve
osha_archive = pd.concat([osha_archive, twelve])

In [487]:
osha_archive

,Company,"Company, City, State, ZIP",Date of Incident,Fatality or Catastrophe,Inspection #,Preliminary Description of Incident,Summary Report Date,Victim(s)
FY 2009,"L.B. Thompson Inc., Tampa, FL 33610",NaN,6/23/2009,NaN,NaN,"Victim was inside a building bundling material and became ill, possible heat stroke.",7/24/2009,NaN
FY 2009,"Presiliana Zamora, Conroe, TX 77304",NaN,6/30/2009,NaN,NaN,Victim was on a scaffold doing repairs on an archway and the scaffold collapsed.,7/24/2009,NaN
FY 2009,"Cesar Tapia, Miami, FL 33166",NaN,7/6/2009,NaN,NaN,"Victim was conducting roofing work, and fell through a hole in the roof, 25 feet to the concrete floor below.",7/24/2009,NaN
FY 2009,"Knobel Refrigeration, Inc., North Platte, NE 69103",NaN,7/9/2009,NaN,NaN,Victim was climbing a ladder while servicing an air-conditioning unit and fell from the ladder.,7/24/2009,NaN
FY 2009,"Greensburg, PA 15601",NaN,7/11/2009,NaN,NaN,"Employees were throwing away old files. They had a dumpster on the forks in the raised position. In bringing the dumpster down on the forks, the load became unstable. Victim stood on the rear of the forklift to act as a counterbalance. The dumpster fell off the trucs, causing forktruck to lunge forward and the victim to strike his head on the overhead guard, falling to the ground.",7/24/2009,NaN
FY 2009,"Lighthouse Electric, Boulder, CO 80301",NaN,7/11/2009,NaN,NaN,Fatality at a bookstore under construction.,7/24/2009,NaN
FY 2009,"Priority Energy Services, LLC, Raleigh, MS 39153",NaN,7/15/2009,NaN,NaN,"Victim was pressure testing a new metering station at a new pipeline construction project, when an explosion occurred.",7/24/2009,NaN
FY 2009,"KM Davis Contracting Co., Inc., Marietta, GA 30008",NaN,7/16/2009,NaN,NaN,Victim was standing on the side of the road and was struck by a car.,7/24/2009,NaN
FY 2009,"Jenkins Electric, Post, TX 79356",NaN,7/18/2009,NaN,NaN,"Victim was working on a pole replacing a transformer, 14,400KV, and was electrocuted.",7/24/2009,NaN
FY 2009,"Lariat Communications, De Kalb, TX 75559",NaN,7/18/2009,NaN,NaN,Victim was installing coax cable hangers down the face of an existing tower and fell 60 to 100 feet.,7/24/2009,NaN


In [488]:
osha_archive.info()
osha_archive.sample(10)

<class 'pandas.core.frame.DataFrame'>
Index: 7385 entries, FY 2009 to FY 2012
Data columns (total 8 columns):
Company                                3618 non-null object
Company, City, State, ZIP              3734 non-null object
Date of Incident                       3618 non-null object
Fatality or Catastrophe                3734 non-null object
Inspection #                           862 non-null object
Preliminary Description of Incident    7352 non-null object
Summary Report Date                    3618 non-null object
Victim(s)                              520 non-null object
dtypes: object(8)
memory usage: 519.3+ KB


,Company,"Company, City, State, ZIP",Date of Incident,Fatality or Catastrophe,Inspection #,Preliminary Description of Incident,Summary Report Date,Victim(s)
2014-05-02 00:00:00,NaN,"True Blue Inc., Orlando, FL 32805",NaN,Fatality,NaN,Worker struck and killed by dump truck.,NaN,NaN
2014-03-20 00:00:00,NaN,"Midwest Farmers Cooperative, Tecumseh, NE 68450",NaN,Fatality,NaN,Worker died from exposure to anhydrous ammonia released during transfer from tanker trailer to bulk storage tank.,NaN,NaN
2014-11-11 00:00:00,NaN,"Fred Carries Contracts Inc., Phoenix, AZ 85006",NaN,Fatality,NaN,Worker fatally crushed when car fell off jack stand during repairs.,NaN,NaN
FY 2010,"GE Transportation, Kansas City, MO� 64153",NaN,03/17/10,NaN,NaN,"Worker was working on the wheel and axle line, using an overhead bridge crane to lift a 5,100-lb locomotive wheel set.� Worker lifted the locomotive wheel set, causing it to swing in a pendulum motion, pinning his leg against another stationary locomotive wheel set.",03/27/10,NaN
2015-08-04 00:00:00,NaN,"Dalton Sealing and Paving, Columbus, OH 43207",NaN,Fatality,1082498,Worker struck and killed by dump truck tailgate.,NaN,Verna Riffle
2015-06-08 00:00:00,NaN,"Commercial Cool-Temp Corp., Homestead, FL 33030",NaN,Fatality,1071299,Worker killed in fall while welding.,NaN,NaN
FY 2012,"V & M Tube Alloy, Houston, TX 77049",NaN,2/1/2012,NaN,NaN,Worker repairing a hydrotester in a pipe manufacturing facility was killed after being struck on the head by a pipe on a conveyor belt.,2/4/2012,NaN
2015-03-24 00:00:00,NaN,"Reliance Well Service Inc., Fairfield, TX 75840",NaN,Fatality,1048906,Worker struck and killed by machine parts when drill line broke.,NaN,NaN
2014-08-02 00:00:00,NaN,"Rebel Oil Co., Inc. Las Vegas, NV 89115",NaN,Fatality,NaN,Worker died from exposure to ethanol vapor in tanker rail car.,NaN,NaN
FY 2010,"Main Street Feeds Inc., Monett, MO 65708",NaN,12/01/09,NaN,NaN,Worker fell from atop of a feed truck.,12/04/09,NaN


In [490]:
# if a row is totally blank, drop it
# osha_archive.dropna(axis=1, how="all" ).info() # inplace=True
# osha_archive[:1]

In [491]:
osha_archive.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7385 entries, FY 2009 to FY 2012
Data columns (total 8 columns):
Company                                3618 non-null object
Company, City, State, ZIP              3734 non-null object
Date of Incident                       3618 non-null object
Fatality or Catastrophe                3734 non-null object
Inspection #                           862 non-null object
Preliminary Description of Incident    7352 non-null object
Summary Report Date                    3618 non-null object
Victim(s)                              520 non-null object
dtypes: object(8)
memory usage: 519.3+ KB


In [492]:
# though we have a date of incident field, it's missing often, and the index sometimes has it
# so make a date col it's own thing, but some are 'FY 09'
osha_archive.reset_index(level=0,  inplace=True)
osha_archive['date'] = osha_archive['index']
osha_archive.drop('index', axis=1, inplace=True)
osha_archive.info()

# date of incident is better than date, it takes presidence 
osha_archive['date'] = osha_archive['Date of Incident'].fillna(osha_archive['date'])
# and then drop the other,
osha_archive.drop('Date of Incident', axis=1, inplace=True)

osha_archive[:1]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7385 entries, 0 to 7384
Data columns (total 9 columns):
Company                                3618 non-null object
Company, City, State, ZIP              3734 non-null object
Date of Incident                       3618 non-null object
Fatality or Catastrophe                3734 non-null object
Inspection #                           862 non-null object
Preliminary Description of Incident    7352 non-null object
Summary Report Date                    3618 non-null object
Victim(s)                              520 non-null object
date                                   7352 non-null object
dtypes: object(9)
memory usage: 519.3+ KB


,Company,"Company, City, State, ZIP",Fatality or Catastrophe,Inspection #,Preliminary Description of Incident,Summary Report Date,Victim(s),date
0,"L.B. Thompson Inc., Tampa, FL 33610",NaN,NaN,NaN,"Victim was inside a building bundling material and became ill, possible heat stroke.",7/24/2009,NaN,6/23/2009


In [ ]:
# osha_archive["Company"].value_counts()


# HERE HERE HERE 💥
- [ ] Fill and drop cols
- [ ] Run geocoder

In [503]:
osha_archive['place'] = osha_archive['Company'].fillna(osha_archive['Company, City, State, ZIP'])
osha_archive.drop(['Company', 'Company, City, State, ZIP'], inplace=True, axis=1)
osha_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7385 entries, 0 to 7384
Data columns (total 7 columns):
Fatality or Catastrophe                3734 non-null object
Inspection #                           862 non-null object
Preliminary Description of Incident    7352 non-null object
Summary Report Date                    3618 non-null object
Victim(s)                              520 non-null object
date                                   7352 non-null object
place                                  7352 non-null object
dtypes: object(7)
memory usage: 403.9+ KB


In [505]:
# osha_archive.place.value_counts()

In [373]:
# this may not be the description, but it does seem to have something valid if there's anything
# osha_archive.dropna(axis=0, how="any", inplace=True, subset=['Preliminary Description of Incident'])

In [ ]:
# len(osha_archive[osha_archive['Unnamed: 4'].notnull()] )
# osha_archive[osha_archive['Unnamed: 4'].notnull()][:1]
#[:3]
# todo, fix this or drop these,
# drop for now, unamed 4 is prelim descrip, and prelim descrip is address for these 100 or so
# osha_archive.info()

## Geocoding the Archive

In [508]:
geo_cache['cache_hit_count'] = 0
geo_cache['cache_miss_count'] = 0

In [509]:

# for some reason doing these together (apply, and then applying pd.Series) says it's the wrong shape, may be a progress apply issue? XXX
# ok apply it
# wait it gets that no matter what, it must be a problem with progress apply or the way i'm doing this or the latlng func
tqdm_pandas(tqdm())
osha_archive_loc = osha_archive.progress_apply (lambda row: get_loc_for_row(row, 'place'), axis=1)


0it [00:00, ?it/s]

2125it [00:00, 21247.49it/s]
4996it [00:00, 23043.28it/s]
6304it [00:01, 3712.29it/s] 
7385it [00:01, 5786.34it/s]

sometimes running above errors and says
```
ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='maps.googleapis.com', port=443): Read timed out. (read timeout=5.0)",), 'occurred at index 5088')
```

and this is the weird shape error it gets, but not if I rerun it once everything is cached
```
ValueError: Shape of passed values is (7352, 2), indices imply (7352, 11)
```

In [510]:
osha_archive_loc = osha_archive_loc.apply(pd.Series)

In [ ]:
osha_archive_loc

In [511]:
print(geo_cache['cache_hit_count'], geo_cache['cache_miss_count'])

9703 1


In [512]:
# osha_archive_loc.apply(pd.Series)
osha_archive['lat']=osha_archive_loc[0]
osha_archive['lng']=osha_archive_loc[1]

# could use date as index, meh
# osha_archive['Date of Incident']= pd.to_datetime(osha_archive['Date of Incident'])
# osha_archive = osha_archive.set_index((osha_archive.select_dtypes(include=[np.datetime64]).columns).tolist())
# osha_archive.sort_index(inplace=True)
# untried, Weds XXX


In [513]:
osha_archive[osha_archive.lat.isnull()]
# get_lat_lng('Terry"s Construction, Blackwell, OK 74631')
# osha_archive[
# osha_archive.dropna(axis=1, how='any').info()
# osha_archive.lat.value_counts()
# len(osha_archive[osha_archive.place.str.contains('worker')]) 
# osha_archive.place.str.contains('worker').value_counts()

,Fatality or Catastrophe,Inspection #,Preliminary Description of Incident,Summary Report Date,Victim(s),date,place,lat,lng
948,NaN,NaN,Worker was operating a propane cook stove and was severely burned after the propane gas system ignited causing a fire and/or explosion.,06/05/10,NaN,06/01/10,"Francois Lessard, T3R13, ME 04462",NaN,NaN
2992,Fatality,NaN,"Worker died from head injuries after vehicle he was repairing dropped, pinning him between cab and front bumper.",NaN,NaN,2013-05-29 00:00:00,"US Army Reserve, 88th Regional Support Command, Fort McCoy, WI 54656",NaN,NaN
3593,Fatality,NaN,Employee died while conducting dive operations at 130 feet.,NaN,NaN,2012-10-10 00:00:00,"Stryker Electrical Contracting Inc., Doral, FL 33178 (US Army - Aberdeen Test Center)",NaN,NaN
3614,Fatality,NaN,Pilot found dead along freeway of apparent homicide.,NaN,NaN,2012-10-02 00:00:00,"Alaska Airlines,Golden Scott Freeway I-5,Scott Exit Road, Burbank, CA 91502",NaN,NaN
3698,Fatality,NaN,Worker performing road construction struck and killed by vehicle.,NaN,NaN,2014-09-19 00:00:00,"J.T. Russell & Sons Inc., NC 18 & I-40/ Exit 105, Westbound, Morganton, NC 28655",NaN,NaN
5573,Fatality,1060481,Worker killed in fall from top of crane cab.,NaN,Arthur Hilario Jr.,2015-05-02 00:00:00,"414 Chaple Street NEW HAVEN O & G Industries, Inc. / Tutor Perini Corporation, a Joint Venture, New Haven, CT 06511",NaN,NaN
7343,NaN,NaN,Worker making repairs to a tire compactor was crushed between the compactor and a drum roller.,9/15/2012,NaN,9/13/2012,"Fisher Industries, Interstate 80 East Bound, Mile Post 392, NV 89521",NaN,NaN


In [515]:
# get_lat_lng(" Mile Post 392, NV 89521")
# get_lat_lng("Blackwell, OK 74631'")
osha_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7385 entries, 0 to 7384
Data columns (total 9 columns):
Fatality or Catastrophe                3734 non-null object
Inspection #                           862 non-null object
Preliminary Description of Incident    7352 non-null object
Summary Report Date                    3618 non-null object
Victim(s)                              520 non-null object
date                                   7352 non-null object
place                                  7352 non-null object
lat                                    7378 non-null float64
lng                                    7378 non-null float64
dtypes: float64(2), object(7)
memory usage: 519.3+ KB


In [ ]:
# osha_archive['Preliminary Description of Incident'].fillna('', inplace=True)
# osha_archive.info()
# same for Company, and apply it, so no error below


In [ ]:
# osha_archive.to_csv('osha_archive_wll.csv')

In [516]:
# just change name to description
osha_archive['description'] = osha_archive['Preliminary Description of Incident']
osha_archive.drop(['Preliminary Description of Incident'], inplace=True, axis=1)
osha_archive[:5]    

,Fatality or Catastrophe,Inspection #,Summary Report Date,Victim(s),date,place,lat,lng,description
0,NaN,NaN,7/24/2009,NaN,6/23/2009,"L.B. Thompson Inc., Tampa, FL 33610",27.994928,-82.400987,"Victim was inside a building bundling material and became ill, possible heat stroke."
1,NaN,NaN,7/24/2009,NaN,6/30/2009,"Presiliana Zamora, Conroe, TX 77304",30.331525,-95.509247,Victim was on a scaffold doing repairs on an archway and the scaffold collapsed.
2,NaN,NaN,7/24/2009,NaN,7/6/2009,"Cesar Tapia, Miami, FL 33166",25.823504,-80.310168,"Victim was conducting roofing work, and fell through a hole in the roof, 25 feet to the concrete floor below."
3,NaN,NaN,7/24/2009,NaN,7/9/2009,"Knobel Refrigeration, Inc., North Platte, NE 69103",41.142954,-100.790990,Victim was climbing a ladder while servicing an air-conditioning unit and fell from the ladder.
4,NaN,NaN,7/24/2009,NaN,7/11/2009,"Greensburg, PA 15601",40.301458,-79.538929,"Employees were throwing away old files. They had a dumpster on the forks in the raised position. In bringing the dumpster down on the forks, the load became unstable. Victim stood on the rear of the forklift to act as a counterbalance. The dumpster fell off the trucs, causing forktruck to lunge forward and the victim to strike his head on the overhead guard, falling to the ground."


In [518]:
# OK THIS IS WHERE YOU STOPPED, now drop archive first, should deal with current up there

osha_archive.drop(['Fatality or Catastrophe', 'Inspection #', 'Summary Report Date', 'Victim(s)'], inplace=True, axis=1)


In [519]:
osha_archive

,date,place,lat,lng,description
0,6/23/2009,"L.B. Thompson Inc., Tampa, FL 33610",27.994928,-82.400987,"Victim was inside a building bundling material and became ill, possible heat stroke."
1,6/30/2009,"Presiliana Zamora, Conroe, TX 77304",30.331525,-95.509247,Victim was on a scaffold doing repairs on an archway and the scaffold collapsed.
2,7/6/2009,"Cesar Tapia, Miami, FL 33166",25.823504,-80.310168,"Victim was conducting roofing work, and fell through a hole in the roof, 25 feet to the concrete floor below."
3,7/9/2009,"Knobel Refrigeration, Inc., North Platte, NE 69103",41.142954,-100.790990,Victim was climbing a ladder while servicing an air-conditioning unit and fell from the ladder.
4,7/11/2009,"Greensburg, PA 15601",40.301458,-79.538929,"Employees were throwing away old files. They had a dumpster on the forks in the raised position. In bringing the dumpster down on the forks, the load became unstable. Victim stood on the rear of the forklift to act as a counterbalance. The dumpster fell off the trucs, causing forktruck to lunge forward and the victim to strike his head on the overhead guard, falling to the ground."
5,7/11/2009,"Lighthouse Electric, Boulder, CO 80301",40.040651,-105.207780,Fatality at a bookstore under construction.
6,7/15/2009,"Priority Energy Services, LLC, Raleigh, MS 39153",32.033488,-89.522291,"Victim was pressure testing a new metering station at a new pipeline construction project, when an explosion occurred."
7,7/16/2009,"KM Davis Contracting Co., Inc., Marietta, GA 30008",33.882159,-84.527742,Victim was standing on the side of the road and was struck by a car.
8,7/18/2009,"Jenkins Electric, Post, TX 79356",33.191044,-101.375653,"Victim was working on a pole replacing a transformer, 14,400KV, and was electrocuted."
9,7/18/2009,"Lariat Communications, De Kalb, TX 75559",33.508726,-94.616319,Victim was installing coax cable hangers down the face of an existing tower and fell 60 to 100 feet.


In [300]:
# ok, time to drop stuff from both

# current first
osha_current.drop([ 'fatcat', 'inspection', 'victim'], axis=1, inplace=True)
osha_current.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573 entries, 0 to 572
Data columns (total 5 columns):
date           571 non-null object
description    573 non-null object
employer       573 non-null object
lat            561 non-null float64
lng            561 non-null float64
dtypes: float64(2), object(3)
memory usage: 22.5+ KB


In [523]:
# osha_archive.info()
osha_current.columns = ['date', 'description', 'place', 'lat', 'lng']
osha_current.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573 entries, 0 to 572
Data columns (total 5 columns):
date           571 non-null object
description    573 non-null object
place          573 non-null object
lat            561 non-null float64
lng            561 non-null float64
dtypes: float64(2), object(3)
memory usage: 22.5+ KB


### Now to combine the current and archive
and spit out our result

In [524]:
osha_all = pd.concat([osha_archive, osha_current], ignore_index=True)

# osha_archive.dropna(how='any', inplace=True)
# osha_archive.info()

In [525]:
osha_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7958 entries, 0 to 7957
Data columns (total 5 columns):
date           7923 non-null object
description    7925 non-null object
lat            7939 non-null float64
lng            7939 non-null float64
place          7925 non-null object
dtypes: float64(2), object(3)
memory usage: 310.9+ KB


In [526]:
osha_all.to_csv('osha_all_all.csv')
# combine all the osha records, before splitting by time below

In [527]:
# there are some with problems
# ?
osha_all.dropna(axis=0, how='any', inplace=True)

In [528]:
osha_all.info()
osha_all[-3:]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7904 entries, 0 to 7955
Data columns (total 5 columns):
date           7904 non-null object
description    7904 non-null object
lat            7904 non-null float64
lng            7904 non-null float64
place          7904 non-null object
dtypes: float64(2), object(3)
memory usage: 370.5+ KB


,date,description,lat,lng,place
7953,2016-07-19,Worker killed in fall through stair hole.,42.544013,-83.346878,"Johni Semma DBA Johni Semma, West Bloomfield, MI 48322"
7954,2016-06-27,Worker died from hantavirus after being exposed to mouse droppings while cleaning cabins.,34.687556,-106.351590,"Marzano Mountain Retreat, Torreon, NM 87061"
7955,2016-05-24,Worker struck and killed by falling tree.,37.683378,-87.916691,"Raymond Trent dba Raymond Trent, Morganfield, KY 42437"


In [529]:
osha_all.to_csv('osha.csv')

In [501]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [502]:
save_obj(geo_cache, "geo_cache")

In [407]:
dfc = pd.DataFrame({'A':['aaa','bbb','ccc'],'B':[1,2,3]})
# dfc.loc[dfc.A.notnull(), 'B'] = dfc.loc[dfc.A.notnull(), 'A'] 

# dfc.loc[0,'A'] = 1188


dfc

,A,B
0,aaa,aaa
1,bbb,bbb
2,ccc,ccc
